In [ ]:
# Balance Data

import numpy as np
print(np.__version__) # Use 1.21.6
import pandas as pd
from collections import Counter
from sklearn.model_selection import train_test_split
from random import shuffle

for v in range(1, 6):
    train_data = np.load(f'raw_data/training_data-{v}.npy', allow_pickle=True)  
    WIDTH = 270
    HEIGHT = 480

    df = pd.DataFrame(train_data)
    print(df.head())
    print(Counter(df[1].apply(str)))

    w = []
    s = []
    a = []
    d = []
    wa = []
    wd = []
    sa = []
    sd = []
    nk = []


    for data in train_data:
        img = data[0]
        choice = data[1]

        if choice == [1, 0, 0, 0, 0, 0, 0, 0, 0]:
            w.append([img, choice])
        elif choice == [0, 1, 0, 0, 0, 0, 0, 0, 0]:
            s.append([img, choice])
        elif choice == [0, 0, 1, 0, 0, 0, 0, 0, 0]:
            a.append([img, choice])
        elif choice == [0, 0, 0, 1, 0, 0, 0, 0, 0]:
            d.append([img, choice])
        elif choice == [0, 0, 0, 0, 1, 0, 0, 0, 0]:
            wa.append([img, choice])
        elif choice == [0, 0, 0, 0, 0, 1, 0, 0, 0]:
            wd.append([img, choice])
        elif choice == [0, 0, 0, 0, 0, 0, 1, 0, 0]:
            sa.append([img, choice])
        elif choice == [0, 0, 0, 0, 0, 0, 0, 1, 0]:
            sd.append([img, choice])
        elif choice == [0, 0, 0, 0, 0, 0, 0, 0, 1]:
            nk.append([img, choice])
        elif choice == [0, 0, 0, 0, 0, 0, 0, 0, 0]:
            pass    
        else:
            print('no matches')    

    print(len(w), len(s), len(a), len(d), len(wa), len(wd), len(sa), len(sd), len(nk))

    # balance the data
    w = w[:len(wa)*2] # Balance W as per need

    # convert to numpy array
    w = np.array(w)
    s = np.array(s)
    a = np.array(a)
    d = np.array(d)
    wa = np.array(wa)
    wd = np.array(wd)
    sa = np.array(sa)
    sd = np.array(sd)
    nk = np.array(nk)

    # Filter out lists with length 0
    non_empty_lists = [lst for lst in [w, s, a, d, wa, wd, sa, sd, nk] if len(lst) > 0]

    # Check if there are any non-empty lists
    if not non_empty_lists:
        print("Error: All lists have length 0.")
    else:
        train_data = np.concatenate(non_empty_lists, axis=0)

    #train_data = np.concatenate((w, s, a, d, wa, wd, sa, sd, nk), axis=0)
    #shuffle(train_data)

    # change shape to (480, 270, 3)
    for i in range(len(train_data)):
        train_data[i][0] = train_data[i][0].reshape(WIDTH, HEIGHT, 3)

    # Save the data
    np.save(f'cleaned_data/balanced_data_lstm-{v}.npy', train_data)
    print(f'balanced_data-{v}.npy saved')

    del train_data

print('Done')

# X = np.array([i[0] for i in train_data]).reshape(-1, WIDTH, HEIGHT, 3)
# y = np.array([i[1] for i in train_data])

# Split data into train and test
#X_train, X_test, y_train, y_test = train_test_split(X, y , test_size=0.1, random_state=42)

# Save the data
# np.save(r'X_train.npy', X_train)
# np.save(r'X_test.npy', X_test)
# np.save(r'y_train.npy', y_train)
# np.save(r'y_test.npy', y_test)

In [ ]:
## Load the Balanced Data and training the model
# Alexnet
import numpy as np
import os
from models import alexnet_sartaj
import tensorboard

EPOCHS = 2
WIDTH = 480
HEIGHT = 270
LR = 1e-3
MODEL_NAME = 'alexnet_color_4096dense_1'

for ep in range(EPOCHS): # For all epochs 
    for v in range(1,6): # Load all 5 versions of the data
        train_data = np.load(f'cleaned_data/balanced_data-{v}.npy', allow_pickle=True)

        # change shape to (480, 270, 3)
        for i in range(len(train_data)):
            train_data[i][0] = train_data[i][0].reshape(WIDTH, HEIGHT, 3)

        # Split into X and y
        shuffle(train_data)    
        X = np.array([i[0] for i in train_data]).reshape(-1, WIDTH, HEIGHT, 3)
        y = np.array([i[1] for i in train_data])

        print(X.shape)
        print(y.shape)

        #Split data into train and test
        X_train, X_test, y_train, y_test = train_test_split(X, y , test_size=0.25, random_state=42)

        # Clear the memory
        del train_data
        del X
        del y

        ## TRAINING THE MODEL
        model = alexnet_sartaj(WIDTH, HEIGHT, LR, input=3, output=9, model_name=MODEL_NAME)

        # Check if model is already trained, if so lead if before training
        if os.path.exists(f'{MODEL_NAME}.meta'):
            model.load(MODEL_NAME)
            print('Model loaded!')
        else:
            print('Model not found!')
            print('Training new model...')    

        model.fit({'input': X_train}, {'targets': y_train}, n_epoch=1, validation_set=({'input': X_test}, {'targets': y_test}),run_id=MODEL_NAME, show_metric=True)
        model.save(MODEL_NAME)

In [ ]:
## Load the Balanced Data and training the model
# Alexnet_LSTM
import numpy as np
import os
from models import lstm_alexnet
import tensorboard

EPOCHS = 2
WIDTH = 480
HEIGHT = 270
LR = 1e-3
MODEL_NAME = 'lstm_alexnet_color_4096_v1'

for ep in range(EPOCHS): # For all epochs 
    for v in range(1,6): # Load all 5 versions of the data
        train_data = np.load(f'cleaned_data/balanced_data_lstm-{v}.npy', allow_pickle=True)

        # change shape to (480, 270, 3)
        for i in range(len(train_data)):
            train_data[i][0] = train_data[i][0].reshape(WIDTH, HEIGHT, 3)

        # Split into X and y
        shuffle(train_data)    
        X = np.array([i[0] for i in train_data]).reshape(-1, WIDTH, HEIGHT, 3)
        y = np.array([i[1] for i in train_data])

        print(X.shape)
        print(y.shape)

        #Split data into train and test
        X_train, X_test, y_train, y_test = train_test_split(X, y , test_size=0.25, random_state=42)

        # Clear the memory
        del train_data
        del X
        del y

        ## TRAINING THE MODEL
        model = alexnet_sartaj(WIDTH, HEIGHT, LR, input=3, output=9, model_name=MODEL_NAME)

        # Check if model is already trained, if so lead if before training
        if os.path.exists(f'{MODEL_NAME}.meta'):
            model.load(MODEL_NAME)
            print('Model loaded!')
        else:
            print('Model not found!')
            print('Training new model...')    

        model.fit({'input': X_train}, {'targets': y_train}, n_epoch=1, validation_set=({'input': X_test}, {'targets': y_test}),run_id=MODEL_NAME, show_metric=True)
        model.save(MODEL_NAME)